<a href="https://colab.research.google.com/github/GuiDecker/VAR-AI/blob/main/VAR-Vigilancia-Ambiental-Resiliente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q google-generativeai
!pip install -U -q ultralytics
# !pip install -U -q geocoder
# !pip install -U -q twilio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.4/756.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.5 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai
from ultralytics import YOLO
# from twilio.rest import Client
# import geocoder

In [ ]:
# Dataset treinado YOLO versão 8.

# model = YOLO("YOLOv8n.pt")
model = YOLO("/content/yolov8n.pt")

# Irá fazer a detecção de pessoas, animais, objetos utilizando visão computacional

# model.predict(source="assets/people-flooading/RS_video.mp4", conf=0.1, save=True, show=True)
model.predict(source="/content/RS_video.mp4", conf=0.7, save=True, show=True)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1302) /content/RS_video.mp4: 384x640 (no detections), 108.9ms
video 1/1 (frame 2/1302) /content/RS_video.mp4: 384x640 (no detections), 107.6ms
video 1/1 (frame 3/1302) /content/RS_video.mp4: 384x640 (no detections), 102.5ms
video 1/1 (frame 4/1302) /content/RS_video.mp4: 384x640 (no detections), 98.0ms
video 1/1 (frame 5/1302) /content/RS_video.mp4: 384x640 (no

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

In [ ]:
import cv2
import os
import shutil

from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# O vídeo comentádo de 10min é um vídeo padrão do sherlock homes, caso queira testa-lo também, sugiro passar a variavél full_video como False
# video_file_name = "https://storage.googleapis.com/generativeai-downloads/data/SherlockJr._10min.mp4"

# Vídeo local de desastre ambiental das enchentes no Rio Grande do Sul
video_file_name = "/content/runs/detect/predict/RS_video.avi"


# Crie ou limpe o diretório de frames de imagens extraídos existente.
FRAME_EXTRACTION_DIRECTORY = "/content/frames"
FRAME_PREFIX = "_frame"
def create_frame_output_dir(output_dir):
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  else:
    shutil.rmtree(output_dir)
    os.makedirs(output_dir)

def extract_frame_from_video(video_file_path):
  print(f"Extraindo {video_file_path} a 1 quadro por segundo. pode levar um tempo...")
  create_frame_output_dir(FRAME_EXTRACTION_DIRECTORY)
  vidcap = cv2.VideoCapture(video_file_path)
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  frame_duration = 1 / fps # Intervalo de tempo entre frames (em segundos)
  output_file_prefix = os.path.basename(video_file_path).replace('.', '_')
  frame_count = 0
  count = 0
  while vidcap.isOpened():
      success, frame = vidcap.read()
      if not success: # Final do video
          break
      if int(count / fps) == frame_count: # Extrai um frame a cada segundo
          min = frame_count // 60
          sec = frame_count % 60
          time_string = f"{min:02d}:{sec:02d}"
          image_name = f"{output_file_prefix}{FRAME_PREFIX}{time_string}.jpg"
          output_filename = os.path.join(FRAME_EXTRACTION_DIRECTORY, image_name)
          cv2.imwrite(output_filename, frame)
          frame_count += 1
      count += 1
  vidcap.release()
  print(f"Extração de frames de vídeo concluída!\n\Extraído: {frame_count} frames")

extract_frame_from_video(video_file_name)


Extracting /content/runs/detect/predict/RS_video.avi at 1 frame per second. This might take a bit...
Completed video frame extraction!

Extracted: 45 frames
Uploading 45 files. This might take a bit...
Uploading: /content/frames/RS_video_avi_frame00:00.jpg...
Uploading: /content/frames/RS_video_avi_frame00:01.jpg...
Uploading: /content/frames/RS_video_avi_frame00:02.jpg...
Uploading: /content/frames/RS_video_avi_frame00:03.jpg...
Uploading: /content/frames/RS_video_avi_frame00:04.jpg...
Uploading: /content/frames/RS_video_avi_frame00:05.jpg...
Uploading: /content/frames/RS_video_avi_frame00:06.jpg...
Uploading: /content/frames/RS_video_avi_frame00:07.jpg...
Uploading: /content/frames/RS_video_avi_frame00:08.jpg...
Uploading: /content/frames/RS_video_avi_frame00:09.jpg...
Uploading: /content/frames/RS_video_avi_frame00:10.jpg...
Uploading: /content/frames/RS_video_avi_frame00:11.jpg...
Uploading: /content/frames/RS_video_avi_frame00:12.jpg...
Uploading: /content/frames/RS_video_avi_fram

In [ ]:

import os

class File:
  def __init__(self, file_path: str, display_name: str = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name
    self.timestamp = get_timestamp(file_path)

  def set_file_response(self, response):
    self.response = response

def get_timestamp(filename):
  """Extrai a contagem de Frames (como um número inteiro) de um nome de arquivo com o formato
     'output_file_prefix_frame00:00.jpg'.
  """
  parts = filename.split(FRAME_PREFIX)
  if len(parts) != 2:
      return None  # Indica que o nome do arquivo pode estar formatado incorretamente
  return parts[1].split('.')[0]

# Processa cada Frame do vídeo no diretório de saída
files = os.listdir(FRAME_EXTRACTION_DIRECTORY)
files = sorted(files)
files_to_upload = []
for file in files:
  files_to_upload.append(
      File(file_path=os.path.join(FRAME_EXTRACTION_DIRECTORY, file)))

#Enviar os arquivos para a API
# Carregando apenas uma parte teste de 10 segundos de arquivos para reduzir o tempo de upload.
# Altere full_video para True para enviar o vídeo inteiro e False para reduzido.
full_video = True

uploaded_files = []
print(f'Uploading {len(files_to_upload) if full_video else 10} files. Isso pode levar um tempo...')

for file in files_to_upload if full_video else files_to_upload[40:50]:
  print(f'Uploading: {file.file_path}...')
  response = genai.upload_file(path=file.file_path)
  file.set_file_response(response)
  uploaded_files.append(file)

print(f"Uploads de arquivos concluídos!\n\nUploaded: {len(uploaded_files)} files")


In [ ]:

# Lista os arquivos enviados na API
for n, f in zip(range(len(uploaded_files)), genai.list_files()):
  print(f.uri)

In [ ]:

  # Criação prompt.
prompt = "Analise o vídeo cheque se o ambiente se passa em algum desastre, acidente ou confusão. E se tiver acontecido alguma enchente, fogo, terremoto, destruição de coisas, faça um passo a passo preciso de ações especificas para ajudar as pessoas dependendo da condição do ambiente. E principalmente analise se tem qualquer sinal de pessoas e descreva como e onde essa pessoa estava no vídeo."

model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

# Faça a solicitação GenerateContent com a estrutura descrita acima.
def make_request(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file.timestamp)
    request.append(file.response)
  return request

# Execução da solicitação LLM.
request = make_request(prompt, uploaded_files)
response = model.generate_content(request,
                                  request_options={"timeout": 600})
print(response.text)